In [ ]:
! pip install wandb

In [3]:
import wandb
from wandb.keras import WandbCallback

In [ ]:
wandb.init(project = 'FashionMnist_pytorch')

In [5]:

config = wandb.config


In [8]:
import torch
import torchvision

In [7]:
def calc_acc(pred,labels):
  _,pred_max = torch.max(preds , 1)
  accuracy = torch.sum(pred_max == labels.data , dtype = torch.float64) / len(preds)
  return accuracy



In [9]:
class MyModel(torch.nn.Module):

  def __init__(self):
    super().__init__()
    self.fc1 = torch.nn.Linear(784,128)
    self.fc2 = torch.nn.Linear(128,10)

  def forward(self,x):
    x = x.reshape((x.shape[0],784))
    x = self.fc1(x)
    x = torch.relu(x)
    x = torch.dropout(x,0.2,train = True)
    x = self.fc2(x)
    x = torch.softmax(x,dim =1)
    return x




In [10]:
device = torch.device('cuda')
model = MyModel()
model = model.to(device)
model.train(True)


MyModel(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [11]:
batch_size = 64
epochs = 25
config.learning_rate = 0.001
lr = config.learning_rate


In [ ]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0),(1))
])

dataset = torchvision.datasets.FashionMNIST('./dataset/',train = True, download = True , transform = transform)
train_data_loader = torch.utils.data.DataLoader(dataset, batch_size = batch_size , shuffle = True)


In [13]:
optimizer = torch.optim.Adam (model.parameters(),lr = lr)
loss_function = torch.nn.CrossEntropyLoss()

In [14]:
for epoch in range(epochs):

  train_loss = 0.0
  train_acc = 0.0 

  for images , labels in train_data_loader:
    images = images.to(device)
    labels = labels.to(device)
    optimizer.zero_grad()

    preds =model(images)
    loss = loss_function(preds , labels)
    loss.backward()

    optimizer.step()

    train_loss += loss
    train_acc += calc_acc(preds , labels)

  total_loss =train_loss/len(train_data_loader)
  total_acc = train_acc/len(train_data_loader)

  print(f'Epoch:{epoch+1} , Loss:{total_loss} , Accuracy:{total_acc}')  



Epoch:1 , Loss:1.7069265842437744 , Accuracy:0.7726212686567164
Epoch:2 , Loss:1.6316126585006714 , Accuracy:0.8364372334754797
Epoch:3 , Loss:1.6176263093948364 , Accuracy:0.8474480277185501
Epoch:4 , Loss:1.6102566719055176 , Accuracy:0.8542277452025586
Epoch:5 , Loss:1.604295015335083 , Accuracy:0.8599913379530917
Epoch:6 , Loss:1.6001436710357666 , Accuracy:0.8638059701492538
Epoch:7 , Loss:1.5961575508117676 , Accuracy:0.8667710554371002
Epoch:8 , Loss:1.5935307741165161 , Accuracy:0.869086487206823
Epoch:9 , Loss:1.5900540351867676 , Accuracy:0.8731176705756929
Epoch:10 , Loss:1.588216781616211 , Accuracy:0.874816764392324
Epoch:11 , Loss:1.5860477685928345 , Accuracy:0.876715751599147
Epoch:12 , Loss:1.5825117826461792 , Accuracy:0.8804137793176973
Epoch:13 , Loss:1.5820209980010986 , Accuracy:0.8804470948827292
Epoch:14 , Loss:1.5805126428604126 , Accuracy:0.8819962686567164
Epoch:15 , Loss:1.5801622867584229 , Accuracy:0.8817464019189766
Epoch:16 , Loss:1.575901985168457 , Acc

In [15]:
torch.save(model.state_dict(),"weights.pth")

In [16]:
import cv2 
import numpy as np


model.eval()

#ankle boot image : class number = 9
image =cv2.imread('/content/image.jpg')
image = cv2.cvtColor(image , cv2.COLOR_BGR2GRAY)
image = cv2.resize(image , (28,28))
tensor = transform (image).unsqueeze(0).to(device)

preds = model(tensor)

preds = preds.cpu().detach().numpy()
output = np.argmax(preds)
output

9